In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

from sklearn import utils




/Users/leosong/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Load Data

EMSLoad2020 = pd.read_csv("/HistoricalEMSHourlyLoad-2020.csv")
EMSLoad2021 = pd.read_csv("/HistoricalEMSHourlyLoad-2021.csv")
EMSLoad2022 = pd.read_csv("/HistoricalEMSHourlyLoad-2022.csv")

WeatherData = pd.read_csv("/sandiego-20200101-20221231.csv")

In [3]:
#Combine EMS Loads
EMSLoads = pd.concat([EMSLoad2020, EMSLoad2021, EMSLoad2022])
EMSLoads.drop(EMSLoads.columns.difference(['Date','HR', 'SDGE']), axis =1, inplace=True)
EMSLoads = EMSLoads.reset_index(drop=True)

In [4]:
#number of hours to include
dataPointsCount = 26304

In [5]:
#transfers the features to a dataframe sutiable for the code
features = []

for i in range(1,dataPointsCount):

    temp = []

    k= True
    for j in WeatherData.columns:
        if k:
            k=False
        else:
            temp.append(WeatherData[j][i-1])   
    
    temp.append(EMSLoads['HR'][i])
    
    day = int(i/24)
    temp.append(day)

    features.append(temp)

In [6]:
#tranfers the dependant variables to a form suitable for the code

demand = []

for i in range(1,dataPointsCount):

    demand.append(int(EMSLoads['SDGE'][i]))
    
demandDf = pd.DataFrame(demand)

In [7]:
#splits the data into test and train
X_train,X_test,y_train,y_test=train_test_split(features,demandDf,test_size=0.4,random_state=100)

In [ ]:
#trains the linear model
logreg= LogisticRegression(max_iter=15000)
logreg.fit(X_train,y_train)

/Users/leosong/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#creates a prediction based on the test model
y_predLr=logreg.predict(X_test)

In [ ]:
#calculates the MAPE
sampleSizeCount = 5000

y=y_test.values.tolist()
accuracyLr=[]
for i in range (sampleSizeCount):
    accuracyLr.append(abs(y_predLr[i]-y[i])/y[i])

numLr = 0
for i in range(sampleSizeCount):
   numLr+=abs(accuracyLr[i][0])
print(numLr/sampleSizeCount)

In [ ]:
#creates a trains the random forest regressor
model_rf = RandomForestRegressor(n_estimators=3000)
model_rf.fit(X_train, y_train)


In [ ]:
#gets a prediction based on the test data
pred_test_rf = model_rf.predict(X_test)

In [ ]:
#calculates the mape
sampleSizeCount = 5000

In [ ]:
y=y_test.values.tolist()
accuracyRf=[]
for i in range (sampleSizeCount):
    accuracyRf.append(abs(pred_test_rf[i]-y[i])/y[i])


In [ ]:
numRf = 0
for i in range(sampleSizeCount):
   numRf+=abs(accuracyRf[i][0])
print(numRf/sampleSizeCount)